# Stochastic facility location 

In [1]:
import gamspy as gp
import gamspy.math as gpm
from gamspy import Sum, Card

import sys
import numpy as np
import pandas as pd

Stochastic model (with data defined by scenarios s)

In [2]:
m = gp.Container(options = gp.Options(relative_optimality_gap=0.,absolute_optimality_gap=0.,seed=101))

i = gp.Set(m,'i',description='facilities',records=[f'f{ind+1}' for ind in range(8)])
j = gp.Set(m,'j',description='customers',records=[f'c{ind+1}' for ind in range(12)])
s = gp.Set(m,'s',description='scenarios',records=[f's{ind+1}' for ind in range(50)])

f = gp.Parameter(m,'f',domain=i,description='Fixed cost of opening facility i')
c = gp.Parameter(m,'c',domain=[i,j],description='Variable cost of shipping one unit from i to j')
u = gp.Parameter(m,'u',domain=i,description='Capacity of facility i')
 
p = gp.Parameter(m,'p',domain=s,description='scenario probability')
d = gp.Parameter(m,'d',domain=[j,s],description='Demand of customer j in scenario s')

m.loadRecordsFromGdx("sfl.gdx", symbol_names=['f','c','u','p','d'])

penaltyCost = gp.Parameter(m,'penaltyCost',
    records=50,description='Unit cost for not meeting demand')

x = gp.Variable(m,'x','binary',domain=i,description='Open  facility i')
y = gp.Variable(m,'y','positive',domain=[i,j,s],description='Amount to ship i to j in s')
e = gp.Variable(m,'e','positive',domain=[j,s],description='Shortage at j in s')

demand_eq = gp.Equation(m,'demand_eq',domain=[j,s])
demand_eq[j,s] = Sum(i, y[i,j,s]) + e[j,s] >=  d[j,s]

cap_eq = gp.Equation(m,'cap_eq',domain=[i,s])
cap_eq[i,s] = Sum(j, y[i,j,s]) <= u[i] * x[i]

sfl= gp.Model(m,'sfl',
    equations=m.getEquations(),
    problem=gp.Problem.MIP,
    sense=gp.Sense.MIN,
    objective=Sum(i, f[i]*x[i]) +
        Sum(s, p[s]*(Sum([i,j], c[i,j]*y[i,j,s]) +
            Sum(j, penaltyCost * e[j,s])))
)

y.up[i,j,s] = d[j,s]
e.fx[j,s] = 0
sfl.solve(solver='cplex',solver_options={'threads': 2, 'lpmethod': 4, 'advind': 0, 'solutiontype': 2},output=None)

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,3264.82348240845,1001,5409,MIP,CPLEX,0.035


In [3]:
display(x.l.records)

,i,level
0,f1,0.0
1,f2,1.0
2,f3,1.0
3,f4,1.0
4,f5,1.0
5,f6,1.0
6,f7,0.0
7,f8,1.0


In [4]:
# Now allow shorting
e.up[j,s] = gp.SpecialValues.POSINF
sfl.solve(solver='cplex',solver_options={'threads': 2, 'lpmethod': 4, 'advind': 0, 'solutiontype': 2},output=None)

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,3099.16958407758,1001,5409,MIP,CPLEX,0.025


In [5]:
display(x.l.records)
display(np.max(e.toDense()))

,i,level
0,f1,1.0
1,f2,0.0
2,f3,1.0
3,f4,1.0
4,f5,1.0
5,f6,1.0
6,f7,0.0
7,f8,1.0


np.float64(14.399156431082197)

# Probabilistic constraint

In [6]:
alpha = gp.Parameter(m,'alpha',description='Probability fail to meet all demand',records=0.1)
z = gp.Variable(m,'z','binary',domain=s,description='fail to satisfy all demand in scenario s')

demand_eq[j,s] = Sum(i, y[i,j,s]) + d[j,s]*z[s] >=  d[j,s]

prob_eq = gp.Equation(m,'prob_eq')
prob_eq[:] = Sum(s, p[s]*z[s]) <= alpha

sflcc= gp.Model(m,'sflcc',
    equations=m.getEquations(),
    problem=gp.Problem.MIP,
    sense=gp.Sense.MIN,
    objective=Sum(i, f[i]*x[i]) +
        Sum(s, p[s]*Sum([i,j], c[i,j]*y[i,j,s]))
)

sflcc.solve(solver='cplex',solver_options={'threads': 2, 'lpmethod': 4, 'advind': 0, 'solutiontype': 2},output=None)

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,2554.56944339029,1003,5460,MIP,CPLEX,0.317


In [7]:
unmetdem = gp.Parameter(m,'unmetdem',domain=s,description='unmet demand')
unmetdem[s] = gp.Smax(j, d[j,s] - Sum(i, y.l[i,j,s]))
display(x.l.records, unmetdem.records.round(4), z.l.records)

,i,level
0,f1,1.0
1,f2,0.0
2,f3,1.0
3,f4,1.0
4,f5,1.0
5,f6,1.0
6,f7,0.0
7,f8,0.0


,s,value
0,s1,-0.0000
1,s2,-0.0000
2,s3,-0.0000
3,s4,-0.0000
4,s5,-0.0000
5,s6,-0.0000
6,s7,194.6691
7,s8,-0.0000
8,s9,-0.0000
9,s10,-0.0000


,s,level
0,s1,0.0
1,s2,0.0
2,s3,0.0
3,s4,0.0
4,s5,0.0
5,s6,0.0
6,s7,1.0
7,s8,0.0
8,s9,0.0
9,s10,0.0
